# Customer Service Flow Analysis 

## Background

All of the data seen below is for the client SharkNinja. They recently launched 5 new flows that presumably survey clients in 5 distinct countries about their customer service experience. 

## Objective

This analysis strives to better understand the flows, when they started, how they have preformed over time, and what they look like from a user's perspective. 

In [1]:
import sys
sys.path.append("..")

import os
from google.cloud import bigquery
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

from src import SankeyFlow




credential_path = "/home/kerri/bigquery-jaya-consultant-cosmic-octane-88917-c46ba9b53a3b.json"
assert os.path.exists(credential_path)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
project_id = 'cosmic-octane-88917'
client = bigquery.Client(project=project_id)

In [2]:
sql_temp = """
WITH major_events AS (
SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', 'API_EVENT') AS ActionId,
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_ApiEvent`

UNION ALL 

SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', 'CALL_EVENT', ':', CallAnswerIndicator) AS ActionId,
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_CallEvent`
WHERE ActionId != 'TRANSFER_33'

UNION ALL 

SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', CAST(DtmfInput AS STRING), '-dtmf') ActionId,
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_DtmfEvent`

UNION ALL 

SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', 'MSG_EVENT') AS ActionId,
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_MsgEvent`

UNION ALL 

SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', 'NLP_EVENT') AS ActionId,
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_NlpEvent`

/*
UNION ALL 


SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', 'EXIT_EVENT') AS ActionId,
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_ApplicationExitEvent`

UNION ALL 


SELECT 
      SessionId,
      TimeStamp,
      CONCAT(ActionId, '~~', 'RECORD_EVENT'),
      CustomerId,
      FlowName
FROM `cosmic-octane-88917.analytics_us._VW_CallDetailRecordEvent`
*/

),

filtered AS (
SELECT  SessionId,
        TimeStamp,
        ActionId,
        FlowName
FROM major_events
WHERE CustomerId IN ('1021814')
--AND FlowName='{0}'
AND EXTRACT(DATE FROM TimeStamp) > '2020-06-01'
),

metric_prep AS (
SELECT 
       SessionId,
       ActionId,
       TimeStamp,
       ROW_NUMBER() OVER (PARTITION BY SessionId ORDER BY TimeStamp) AS rank_event,
       FIRST_VALUE(TimeStamp) OVER (PARTITION BY SessionId ORDER BY TimeStamp) AS first_value,
       LEAD(TimeStamp) OVER (PARTITION BY SessionId ORDER BY TimeStamp) AS next_value,
       LEAD(ActionId) OVER (PARTITION BY SessionId ORDER BY TimeStamp) AS next_event,
       FlowName,
       
FROM
filtered
)

SELECT
       SessionId AS user_id,
       ActionId  AS event_name,
       TimeStamp AS time_event,
       rank_event,
       next_event,
       TIMESTAMP_DIFF(TimeStamp, first_value, SECOND) AS time_from_start,
       TIMESTAMP_DIFF(next_value, TimeStamp, SECOND) AS time_to_next
FROM metric_prep
--WHERE ActionId NOT LIKE '%END_CALL%'
ORDER BY user_id, time_event
LIMIT 100000
"""


In [3]:
sql_time_evol = """
WITH count_steps AS (
SELECT
        TimeStamp,
        SessionId,
        FlowName,
        CHARACTER_LENGTH(Path) - CHARACTER_LENGTH(REPLACE(Path, ';', '')) AS pathLength
FROM `cosmic-octane-88917.analytics_us._VW_ApplicationExitEvent`
),

calc_values AS (
SELECT 
      EXTRACT(DATE FROM TimeStamp) AS date,
      COUNT(DISTINCT(SessionId)) AS count,
      AVG(pathLength) AS avg_pathLength,
      FlowName
FROM count_steps
WHERE FlowName in {0}
GROUP BY date, FlowName

)

SELECT *,
      AVG(count) OVER(PARTITION BY FlowName ORDER BY date 
                 ROWS BETWEEN 14 PRECEDING AND CURRENT ROW) AS avg_14_day_count,
      AVG(avg_pathLength) OVER(PARTITION BY FlowName ORDER BY date 
                 ROWS BETWEEN 14 PRECEDING AND CURRENT ROW) AS avg_14_day_pathLength
FROM calc_values

"""


In [4]:


def time_stats(flownames, hue):
    
    q = sql_time_evol.format(flownames)
    df = client.query(q).to_dataframe()
    
    fig, (ax1,ax2, ax3, ax4) = plt.subplots(nrows=4, figsize=(15,30))
    chart = sns.lineplot(x="date", y="avg_14_day_count",
                 hue=hue, 
                 data=df, ax=ax1)
    chart.set_title(f"14 Day Rolling Average Count")


    chart = sns.lineplot(x="date", y="avg_14_day_pathLength",
                 hue=hue, 
                 data=df, ax=ax2)
    chart.set_title(f"14 Day Rolling Avg Path Length")


    chart = sns.lineplot(x="date", y="count",
                 hue=hue, 
                 data=df, ax=ax3)
    chart.set_title(f"Count")


    chart = sns.lineplot(x="date", y="avg_pathLength",
                 hue=hue, 
                 data=df, ax=ax4)
    chart.set_title(f"Path Length")

In [5]:
flows = ["United Kingdom - Customer Service", 
         "Italy - Customer Service",
         "France - Customer Service",
         "Spain - Customer Service",
         "Germany - Customer Service"]
#time_stats(tuple(flows), 'FlowName')

# Preformance Over Time

The plots compare the 5 flows over two dimensions. 

1. Count - The number of unique SessionIds
2. PathLength - The number of steps present in the ApplicationExitEvent path variable

### Takeaways

- United Kingdom has a much higher volume than the other flows
- United Kingdom has a higher and more stable average path length
- Path lengths differ between weekdays and weekend, probably because SharkNinja is not taking calls on weekends
- The total count of people calling in on weekends is also smaller
- While path lenght for United Kingdom and Germany became stable around June Italy, France, and Spain did not really level out until September.

In [ ]:
FlowName =  "United Kingdom - Customer Service"
df = client.query(sql_temp.format(FlowName)).to_dataframe()
print(f"Data Size: {len(df)}")
uk_flow = SankeyFlow(FlowName, df)
uk_flow.plot(500, f"{FlowName} Without Trimming").show()

/home/kerri/Documents/Projects/intelepeer/sharkninja_analysis/venv8/lib/python3.8/site-packages/google/cloud/bigquery/client.py:439: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


Data Size: 100000


In [ ]:
df[df['user_id'] == '1591615402-1021814-SR-000-0000000000000-7FE956AC']

In [ ]:
uk_flow.plot(5000, f"{FlowName} With Trimming").show()

## United Kingdom - Customer Service

The upper plot shows all shows all the plots that a SessionId has taken through the flow from June 1st to today. The lower plot shows a trimmed version with only those paths that have over 100 users. 

### Takeaways
- They appear to enter from two main sources MENU_TREE_17 (MainMenu) or MENU_TREE_6
- The paths from the two entrances are always seperate, it's to say they do not share any nodes

In [ ]:
FlowName = "Italy - Customer Service"
df = client.query(sql_temp.format(FlowName)).to_dataframe()
it_flow = SankeyFlow(FlowName, df)
it_flow.plot(0, f"{FlowName} Without Trimming").show()

In [ ]:
it_flow.plot(5000, f"{FlowName} With Trimming").show()

## Italy - Customer Service

The upper plot shows all shows all the plots that a SessionId has taken through the flow from June 1st to today. The lower plot shows a trimmed version with only those paths that have over 10 users.

## Takeaways


In [ ]:
FlowName = "France - Customer Service"
df = client.query(sql_temp.format(FlowName)).to_dataframe()
fr_flow = SankeyFlow(FlowName, df)
fr_flow.plot(0, f"{FlowName} Without Trimming").show()

In [ ]:
fr_flow.plot(5000, f"{FlowName} With Trimming").show()

## France - Customer Service

The upper plot shows all shows all the plots that a SessionId has taken through the flow from June 1st to today. The lower plot shows a trimmed version with only those paths that have over 10 users.

## Takeaways


In [ ]:
FlowName = "Spain - Customer Service"
df = client.query(sql_temp.format(FlowName)).to_dataframe()
es_flow = SankeyFlow(FlowName, df)
es_flow.plot(0, f"{FlowName} Without Trimming").show()

In [ ]:
es_flow.plot(5000, f"{FlowName} With Trimming").show()

## Spain - Customer Service

The upper plot shows all shows all the plots that a SessionId has taken through the flow from June 1st to today. The lower plot shows a trimmed version with only those paths that have over 10 users.

## Takeaways
- The dominate flows ends in call hangup
- Skipping the DTMF_29 menu saves ~10 seconds 
- 41.6k  of

In [ ]:
FlowName = "Germany - Customer Service"
df = client.query(sql_temp.format(FlowName)).to_dataframe()
gr_flow = SankeyFlow(FlowName, df)
gr_flow.plot(0, f"{FlowName} Without Trimming").show()

In [ ]:
gr_flow.plot(6000, f"{FlowName} With Trimming").show()

## Germany - Customer Service

The upper plot shows all shows all the plots that a SessionId has taken through the flow from June 1st to today. The lower plot shows a trimmed version with only those paths that have over 10 users.

## Takeaways


In [ ]:
hungup_events = df[(df['event_name'] == 'ICALL_1~~CALL_EVENT:HUNGUP') & 
                   (df['next_event'] == 'TRANSFER_33~~CALL_EVENT:HUNGUP') &
                   (df['time_from_start'] == '00:08:42')]

In [ ]:
hungup_events

In [ ]:
df[df['user_id'] == '1593587204-1021814-SR-000-0000000000000-28794727']